# 🚨 DDL: Tabela Silver - `tb_chamado_log_invalidos` (Quarentena)

**Origem:** Processamento de Qualidade sobre `chamados_hora`
**Finalidade:** Auditoria e Rejeição

## ⚙️ Estrutura de Qualidade

Esta tabela armazena registros que falharam nas regras de negócio ou integridade referencial da Silver. Não houve troca de tipos significativa, mas sim a **adição de colunas de flag**.

### Flags de Qualidade Adicionadas
Para cada registro rejeitado, persistimos o motivo:
- **`flag_chamado_valido`**: Verifica se o `cd_chamado` não é nulo.
- **`flag_cliente_valido`**: Verifica integridade do cliente.
- **`flag_timestamps_validos`**: Verifica consistência temporal (ex: `dh_fim` não pode ser menor que `dh_inicio`).
- **`flag_qualidade`**: Consolida o status (Geralmente 'ERRO').

### Constraints
- **Primary Key:** `pk_chamado_log_invalidos` (`cd_chamado`). Mantemos a PK para permitir deduplicação mesmo na tabela de erro.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_chamado_log_invalidos"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

In [0]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    cd_chamado BIGINT COMMENT 'Referência ao chamado principal.',
    cd_cliente STRING COMMENT 'Referência ao cliente.',
    dh_abertura TIMESTAMP COMMENT 'Data e hora de abertura do chamado.',
    dh_inicio TIMESTAMP COMMENT 'Data e hora de início do atendimento.',
    dh_fim TIMESTAMP COMMENT 'Data e hora de finalização do atendimento.',
    dt_ingestion TIMESTAMP COMMENT 'Data e hora da ingestão do registro.',
    dc_origem STRING COMMENT 'Identificação do sistema ou arquivo fonte dos dados.',
    flag_chamado_valido BOOLEAN COMMENT 'Indica se o ID do chamado é válido.',
    flag_cliente_valido BOOLEAN COMMENT 'Indica se o ID do cliente é válido.',
    flag_timestamps_validos BOOLEAN COMMENT 'Indica se os timestamps foram convertidos com sucesso.',
    flag_qualidade STRING COMMENT 'Status de qualidade: OK ou ERRO.',
    CONSTRAINT pk_chamado_log_invalidos PRIMARY KEY (cd_chamado)
)
COMMENT 'Tabela silver de auditoria - registros de logs de chamados rejeitados por problemas de qualidade';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")